In [2]:
import openai
from dotenv import load_dotenv, find_dotenv
import os

In [3]:
_ = load_dotenv(find_dotenv()) 
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
MODEL = "gpt-4"

- 담론 구성
    - zero-shot

In [5]:

CONTENT = """
각각의 키워드를 이용하여 이슈에 대한 담론을 만들어라.
이슈 : 의대 정원 확대
긍정입장 대표 키워드: 찬성, 특권, 이기적
부정입장 대표 키워드: 반대, 파업, 피부과

긍정 담론 :
부정 담론 :
"""


In [8]:
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "너는 이슈와 이슈를 잘 나타내는 대표 키워드를 기반으로 이슈에 대한 반론을 친근한 반말 뉴스레터로 만드는 작가야."},
        {"role": "user", "content": CONTENT},
    ],
    temperature=0,
)

print(response['choices'][0]['message']['content'])

긍정 담론 :
야, 너도 알겠지만 지금 의대 정원 확대 이슈가 크게 떠오르고 있어. 그런데 이걸 왜 찬성해야 하는지 아직 모르겠다고? 그럼 내가 좀 알려줄게. 

첫 번째로, 의대 정원 확대는 의사들의 특권을 깨는 거야. 의사가 되려면 얼마나 많은 시간과 노력을 투자해야 하는지 알지? 그런데 그런 노력에 비해 의사들이 받는 대우는 너무 과하다고 생각하지 않아? 의대 정원을 확대하면, 의사들의 특권이 깨지면서 더 많은 사람들이 의사가 될 기회를 얻게 되는 거야.

두 번째로, 의대 정원 확대는 의사들의 이기적인 행동을 막을 수 있어. 의사들이 파업을 하면서 환자들을 피해 보는 일이 없도록 하는 거지. 의대 정원을 확대하면, 의사들이 파업을 해도 환자들에게 큰 피해가 가지 않게 될 거야.

부정 담론 :
하지만, 의대 정원 확대에 반대하는 사람들도 많아. 그 이유를 알고 싶다고? 그럼 내가 좀 알려줄게.

첫 번째로, 의대 정원 확대는 의사들의 파업을 부추기는 거야. 의사들이 파업을 하면서 환자들에게 피해를 주는 건 알겠지? 그런데 의대 정원을 확대하면, 의사들이 파업을 해도 큰 문제가 없다고 생각할 수 있어. 그럼 의사들이 더 자주 파업을 하게 되는 거지.

두 번째로, 의대 정원 확대는 피부과 같은 특정 과에만 의사들이 몰리게 만드는 거야. 의대 정원을 확대하면, 의사가 되기 쉬워지니까 피부과 같은 수익이 높은 과에만 의사들이 몰리게 되는 거야. 그럼 다른 과의 의사들은 부족해지고, 그로 인해 환자들이 피해를 보게 되는 거지.


In [10]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [28]:
response_schemas = [
    ResponseSchema(name="positive_article", description="긍정 담론에 대한 뉴스레터"),
    ResponseSchema(name="negative_article", description="부정 담론에 대한 뉴스레터")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"positive_article": string  // 긍정 담론에 대한 뉴스레터\n\t"negative_article": string  // 부정 담론에 대한 뉴스레터\n}\n```'

In [26]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

#시스템 역할 지정하기
template = "너는 이슈와 이슈를 잘 나타내는 대표 키워드를 기반으로 이슈에 대한 반론을 친근한 반말 뉴스레터로 만드는 작가야."

system_message_prompt = SystemMessagePromptTemplate.from_template(template)


human_template = """
각각의 키워드를 이용하여 이슈에 대한 담론을 만들어라. 
이슈 : {issue_name} 
긍정입장 대표 키워드: {positive_keyword}
부정입장 대표 키워드: {negative_keyword}
{format_instructions}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

query = chat_prompt.format_messages(output_language="Korean",
                            issue_name="의대 정원 확대",
    positive_keyword=["찬성", "특권", "이기적"],
    negative_keyword=["반대", "파업", "피부과"],
    format_instructions=format_instructions
)
llm = ChatOpenAI(temperature=0,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-4',  # 모델명
                )
ans = llm.predict_messages(query)

In [27]:
ans

AIMessage(content='{\n"positive_paragraph": "야, 너도 알겠지만, 우리나라 의대 정원 확대 이슈가 요즘 뜨거워. 많은 사람들이 \'찬성\'하는 이유는 뭐냐면, 의사들의 \'특권\'을 깨기 위해서야. 의사들이 너무 \'이기적\'으로 행동하고 있어서, 그게 사회적 불평등을 만들어내고 있거든. 의대 정원을 확대하면, 의사들의 특권을 깨고, 더 많은 사람들이 의사가 될 수 있어. 그럼 의료 서비스가 더 좋아지고, 의료비도 낮아질 수 있지 않겠냐? 그래서 나는 의대 정원 확대를 찬성하는 거야.",\n\n"negative_paragraph": "하지만, 다른 사람들은 이 의대 정원 확대에 \'반대\'하고 있어. 왜냐하면, 의대 정원 확대가 의사들의 \'파업\'을 초래하고 있거든. 의사들이 파업을 하면, 환자들이 피해를 보는 건 아는거지? 그리고, 의대 정원 확대가 \'피부과\' 같은 특정 전문과를 선호하는 문제도 있어. 의대 정원을 확대하면, 의사들이 피부과 같은 고수익 전문과를 선호하게 되고, 그럼 다른 전문과 의사들이 부족해지는 문제가 생기지 않겠냐? 그래서 나는 의대 정원 확대에 반대하는 거야."\n}')

In [20]:
from langchain.chat_models import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(temperature=0,               # 창의성 (0.0 ~ 2.0) 
                 max_tokens=2048,             # 최대 토큰수
                 model_name='gpt-4',  # 모델명
                )


from langchain.chains import LLMChain

# 질의
print(f'[답변]: {llm.predict(final_prompt)}')
# chatchain = LLMChain(llm=llm, prompt=final_prompt)
# chatchain.run(input_language="English", output_language="Korean")


ValidationError: 1 validation error for HumanMessage
content
  str type expected (type=type_error.str)